In [1]:
import sys
sys.path.append('../')
from utils import *
from model import *

In [2]:
data_dir = '/datadrive/data_bert/'
batch_size = 256
batch_num  = 128
epoch_num  = 200
samp_num   = 7

device = torch.device("cuda:0")
graph = dill.load(open(data_dir + 'graph.pk', 'rb'))

In [3]:
train_range = {t: True for t in graph.times if t != None and t <= 2015}
valid_range = {t: True for t in graph.times if t != None and (t > 2015) & (t < 2018)}
test_range  = {t: True for t in graph.times if t != None and t >= 2018}

In [4]:
def pf_sample(seed, papers, pairs, t_range, batch_size, test = False):
    np.random.seed(seed)
    _time = np.random.choice(list(papers.keys()))
    pids = np.array(papers[_time])[np.random.choice(len(papers[_time]), batch_size, replace = False)]
    fids = []
    edge = defaultdict(lambda: {})
    for x_id, p_id in enumerate(pids):
        f_ids = pairs[p_id]
        for f_id in f_ids:
            if f_id not in fids:
                fids += [f_id]
            edge[x_id][fids.index(f_id)] = True
    pids = np.stack([pids, np.repeat([_time], batch_size)]).T
    fids = np.stack([fids, np.repeat([_time], len(fids))]).T
 
    feature, times, edge_list = sample_subgraph(graph, t_range, \
                inp = {'paper': pids, 'field': fids}, sampled_depth = 4, sampled_number = 128)

    el = []
    for i in edge_list['paper']['field']['rev_PF_in']:
        if i[0] in edge and i[1] in edge[i[0]]:
            continue
        el += [i]
    edge_list['paper']['field']['rev_PF_in'] = el

    el = []
    for i in edge_list['field']['paper']['PF_in']:
        if i[1] in edge and i[0] in edge[i[1]]:
            continue
        el += [i]
    edge_list['field']['paper']['PF_in'] = el
    
    
    node_feature, node_type, edge_time, edge_index, edge_type, node_dict, edge_dict = \
            to_torch(feature, times, edge_list)
    '''
        Trace the paper_id and field_id by its own index plus the type start index
    '''
    paper_ids = np.arange(len(pids)) + node_dict['paper'][0]
    field_ids = np.arange(len(fids)) + node_dict['field'][0]
    ylabel = torch.zeros(batch_size, len(ids))
    for x_id, p_id in enumerate(pids[:,0]):
        for f_id in pairs[p_id]:
            ylabel[x_id][list(ids).index(f_id)] = 1
    ylabel /= ylabel.sum(axis=1).view(-1, 1)
    return node_feature, node_type, edge_time, edge_index, edge_type, field_ids, paper_ids, ylabel
    
def prepare_data(pool, process_ids):
    jobs = []
    for process_id in process_ids[:-1]:
        p = pool.apply_async(pf_sample, args=(np.random.randint(2**32 - 1), train_papers, \
                                               train_pairs, train_range, batch_size))
        jobs.append(p)
    p = pool.apply_async(pf_sample, args=(np.random.randint(2**32 - 1), valid_papers, \
                                           valid_pairs, valid_range, batch_size))
    jobs.append(p)
    return jobs

In [5]:
class RGCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels, num_relations, num_bases, **kwargs):
        super(RGCNConv, self).__init__(aggr='mean', **kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.num_relations = num_relations
        self.num_bases = num_bases
        
        self.ws    = nn.Parameter(torch.Tensor(num_relations, in_channels, out_channels))
        self.bs    = nn.Parameter(torch.Tensor(num_relations, out_channels))
        self.reset_parameters()

    def reset_parameters(self):
        size = self.num_bases * self.in_channels
        uniform(size, self.ws)
        uniform(size, self.bs)


    def forward(self, x, edge_index, edge_type, edge_norm=None, size=None):
        """"""
        return self.propagate(edge_index, size=size, x=x, edge_type=edge_type,
                              edge_norm=edge_norm)


    def message(self, x_j, edge_index_j, edge_type, edge_norm):
        data_size = edge_index_j.size(0)
        res = torch.zeros(data_size, self.out_channels).to(x_j.device)
        
        for r_id in range(self.num_relations):
            idx = (edge_type == r_id)
            if idx.sum() == 0:
                continue
            batch_x_j = x_j[idx]
            res[idx] = torch.mm(batch_x_j, self.ws[r_id]).squeeze(-2) + self.bs[r_id]
        return res if edge_norm is None else res * edge_norm.view(-1, 1)

    def update(self, aggr_out):
        return aggr_out

    def __repr__(self):
        return '{}({}, {}, num_relations={})'.format(
            self.__class__.__name__, self.in_channels, self.out_channels,
            self.num_relations)
    
    
class GNN(nn.Module):
    def __init__(self, in_dim, n_hid, num_relations, n_layers, dropout = 0.3):
        super(GNN, self).__init__()
        self.gcs = nn.ModuleList()
        self.num_relations = num_relations
        self.in_dim    = in_dim
        self.n_hid     = n_hid
        self.drop  = nn.Dropout(dropout)
        self.adapt = nn.Linear(in_dim, n_hid)
        for l in range(n_layers):
            self.gcs.append(RGCNConv(n_hid, n_hid, num_relations, num_relations + int(math.sqrt(n_hid))))
    def set_device(self, device):
        self.device = device
        for gc in self.gcs:
            gc.device = device
    def forward(self, node_feature, edge_index, node_type, edge_type):
        meta_xs = self.drop(F.relu(self.adapt(node_feature)))
        for gc in self.gcs:
            meta_xs = self.drop(F.relu(gc(meta_xs, edge_index, edge_type)))
        return meta_xs

In [6]:
'''
Paper-Field
'''
field_dict = dill.load(open(data_dir + 'field_dict.pk', 'rb'))
ids = np.array([graph.node_forward['field'][k] for k in field_dict if field_dict[k][0] == 'L1'])

paper_ser = {}

train_pairs = {}
valid_pairs = {}
test_pairs  = {}

train_papers = {_time: {} for _time in train_range}
valid_papers = {_time: {} for _time in valid_range}
test_papers  = {_time: {} for _time in test_range}

for f_id in ids:
    for p_id in graph.edge_list['field']['paper']['PF_in'][f_id]:
        _time = graph.edge_list['field']['paper']['PF_in'][f_id][p_id]
        if _time in train_range:
            if p_id not in train_pairs:
                train_pairs[p_id] = []
            train_pairs[p_id] += [f_id]
            train_papers[_time][p_id] = True
        elif _time in valid_range:
            if p_id not in valid_pairs:
                valid_pairs[p_id] = []
            valid_pairs[p_id] += [f_id]
            valid_papers[_time][p_id] = True
        else:
            if p_id not in test_pairs:
                test_pairs[p_id] = []
            test_pairs[p_id] += [f_id]
            test_papers[_time][p_id] = True
for _time in list(train_papers.keys()):
    if len(train_papers[_time]) < batch_size:
        train_papers.pop(_time)
    else:
        train_papers[_time] = np.array(list(train_papers[_time].keys()))
for _time in list(valid_papers.keys()):
    if len(valid_papers[_time]) < batch_size:
        valid_papers.pop(_time)
    else:
        valid_papers[_time] = np.array(list(valid_papers[_time].keys()))
for _time in list(test_papers.keys()):
    if len(test_papers[_time]) < batch_size:
        test_papers.pop(_time)
    else:
        test_papers[_time] = np.array(list(test_papers[_time].keys()))

In [7]:
gnn = GNN(in_dim = len(graph.node_feature['paper']['emb'][0]), n_hid = 400, n_layers = 4, \
          num_relations = len(graph.get_meta_graph()) + 1).to(device)
classifier = Classifier(400, len(ids)).to(device)
model = nn.Sequential(gnn, classifier)
optimizer = torch.optim.AdamW(model.parameters())
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 1000, eta_min=1e-6)

In [ ]:
stats = []
pool = mp.Pool(4)
process_ids = np.arange(batch_num // 4)
st = time.time()
jobs = prepare_data(pool, process_ids)
train_step = 1500
best_val   = 0
res = []
criterion = nn.KLDivLoss(reduction='batchmean')
for epoch in np.arange(epoch_num)+1:
    '''
        Prepare Training and Validation Data
    '''
    train_data = [job.get() for job in jobs[:-1]]
    valid_data = jobs[-1].get()
    pool.close()
    pool.join()
    pool = mp.Pool(4)
    jobs = prepare_data(pool, process_ids)
    et = time.time()
    print('Data Preparation: %.1fs' % (et - st))
    
    
    model.train()
    train_losses = []
    for batch in np.arange(2):
        for node_feature, node_type, edge_time, edge_index, edge_type, field_ids, paper_ids, ylabel in train_data:
            node_rep = gnn.forward(node_feature.to(device), edge_index.to(device), node_type.to(device), edge_type.to(device))
            res  = classifier.forward(node_rep[paper_ids])
            loss = criterion(res, ylabel.to(device))
            optimizer.zero_grad() 
            loss.backward()
            optimizer.step()
            train_losses += [loss.cpu().detach().tolist()]
            train_step += 1
            scheduler.step(train_step)
    '''
        Valid
    '''
    model.eval()
    with torch.no_grad():
        node_feature, node_type, edge_time, edge_index, edge_type, field_ids, paper_ids, ylabel = valid_data
        node_rep = gnn.forward(node_feature.to(device), edge_index.to(device), node_type.to(device), edge_type.to(device))
        res  = classifier.forward(node_rep[paper_ids])
        loss = criterion(res, ylabel.to(device))
        valid_res = []

        for ai, bi in zip(ylabel, res.argsort(descending = True)):
            valid_res += [ai[bi].tolist()]
        valid_ndcg = np.average([ndcg_at_k(resi, len(resi)) for resi in valid_res])
        if valid_ndcg > best_val:
            best_val = valid_ndcg
            torch.save(model, './save/rgcn.pt')
        st = time.time()
        print(("Epoch: %d (%.1fs)  LR: %.5f Train Loss: %.2f  Valid Loss: %.2f  Valid NDCG: %.4f") % \
              (epoch, (st-et), optimizer.param_groups[0]['lr'], np.average(train_losses), loss.cpu().detach().tolist(),\
              valid_ndcg))
        stats += [[np.average(train_losses), loss.cpu().detach().tolist()]]
        if epoch % 5 == 0:
            '''
                Test
            '''
            _time = np.random.choice(list(test_papers.keys()))
            node_feature, node_type, edge_time, edge_index, edge_type, field_ids, paper_ids, ylabel = pf_sample(np.random.randint(2 ** 32 - 1), test_papers, \
                                                           test_pairs, test_range, batch_size, test=True)
            paper_rep = gnn.forward(node_feature.to(device), edge_index.to(device), node_type.to(device), edge_type.to(device))[paper_ids]
            res = classifier.forward(paper_rep)
            test_res = []
            for ai, bi in zip(ylabel, res.argsort(descending = True)):
                test_res += [ai[bi].tolist()]
            test_ndcg = np.average([ndcg_at_k(resi, len(resi)) for resi in test_res])
            print(test_ndcg)
    del train_data, valid_data

Data Preparation: 153.5s


/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type GNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type RGCNConv. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch: 1 (90.8s)  LR: 0.00060 Train Loss: 3.40  Valid Loss: 2.80  Valid NDCG: 0.4954
Data Preparation: 85.8s
Epoch: 2 (91.0s)  LR: 0.00069 Train Loss: 2.93  Valid Loss: 2.94  Valid NDCG: 0.3993
Data Preparation: 88.3s
Epoch: 3 (101.2s)  LR: 0.00078 Train Loss: 2.87  Valid Loss: 3.35  Valid NDCG: 0.4137
Data Preparation: 83.5s
Epoch: 4 (91.5s)  LR: 0.00085 Train Loss: 2.76  Valid Loss: 3.49  Valid NDCG: 0.4022
Data Preparation: 84.8s
Epoch: 5 (84.7s)  LR: 0.00091 Train Loss: 2.70  Valid Loss: 4.03  Valid NDCG: 0.3655
0.33703586357263027
Data Preparation: 86.6s
Epoch: 6 (88.2s)  LR: 0.00096 Train Loss: 2.55  Valid Loss: 3.45  Valid NDCG: 0.3800
Data Preparation: 86.6s
Epoch: 7 (90.6s)  LR: 0.00099 Train Loss: 2.51  Valid Loss: 3.57  Valid NDCG: 0.3974
Data Preparation: 84.0s
Epoch: 8 (86.1s)  LR: 0.00100 Train Loss: 2.41  Valid Loss: 4.36  Valid NDCG: 0.3500
Data Preparation: 82.5s
Epoch: 9 (89.9s)  LR: 0.00099 Train Loss: 2.38  Valid Loss: 3.72  Valid NDCG: 0.4191
Data Preparation: 78.5

/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type GNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type RGCNConv. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch: 12 (90.7s)  LR: 0.00086 Train Loss: 2.24  Valid Loss: 3.04  Valid NDCG: 0.5142
Data Preparation: 77.2s
Epoch: 13 (85.9s)  LR: 0.00079 Train Loss: 2.15  Valid Loss: 3.94  Valid NDCG: 0.4827
Data Preparation: 91.7s
Epoch: 14 (87.4s)  LR: 0.00070 Train Loss: 2.04  Valid Loss: 6.05  Valid NDCG: 0.3595
Data Preparation: 80.5s
Epoch: 15 (86.6s)  LR: 0.00061 Train Loss: 2.04  Valid Loss: 4.28  Valid NDCG: 0.4269
0.401709604465349
Data Preparation: 82.2s
Epoch: 16 (86.8s)  LR: 0.00051 Train Loss: 2.02  Valid Loss: 8.49  Valid NDCG: 0.3081
Data Preparation: 85.3s
Epoch: 17 (83.5s)  LR: 0.00042 Train Loss: 1.93  Valid Loss: 4.48  Valid NDCG: 0.3613


In [ ]:
stats = np.array(stats)
plt.plot(stats[:,0])
plt.plot(stats[:,1])
plt.show()

In [ ]:
model.eval()
gnn, classifier = model
with torch.no_grad():
    test_res = []
    for _ in range(100):
        _time = np.random.choice(list(test_papers.keys()))
        node_feature, node_type, edge_time, edge_index, edge_type, field_ids, paper_ids, ylabel = pf_sample(np.random.randint(2 ** 32 - 1), test_papers, \
                                                       test_pairs, test_range, batch_size, test=True)
        paper_rep = gnn.forward(node_feature.to(device), edge_index.to(device), node_type.to(device), edge_type.to(device))[paper_ids]
        res = classifier.forward(paper_rep)
        for ai, bi in zip(ylabel, res.argsort(descending = True)):
            test_res += [ai[bi].tolist()]
    test_ndcg = [ndcg_at_k(resi, len(resi)) for resi in test_res]
    print(np.average(test_ndcg), np.var(test_ndcg))
    test_mrr = mean_reciprocal_rank(test_res)
    print(np.average(test_mrr), np.var(test_mrr))

In [ ]:
best_model = torch.load('./save/rgcn.pt')

In [ ]:
best_model.eval()
gnn, classifier = best_model
with torch.no_grad():
    test_res = []
    for _ in range(100):
        _time = np.random.choice(list(test_papers.keys()))
        node_feature, node_type, edge_time, edge_index, edge_type, field_ids, paper_ids, ylabel = pf_sample(np.random.randint(2 ** 32 - 1), test_papers, \
                                                       test_pairs, test_range, batch_size, test=True)
        paper_rep = gnn.forward(node_feature.to(device), edge_index.to(device), node_type.to(device), edge_type.to(device))[paper_ids]
        res = classifier.forward(paper_rep)
        for ai, bi in zip(ylabel, res.argsort(descending = True)):
            test_res += [ai[bi].tolist()]
    test_ndcg = [ndcg_at_k(resi, len(resi)) for resi in test_res]
    print(np.average(test_ndcg), np.var(test_ndcg))
    test_mrr = mean_reciprocal_rank(test_res)
    print(np.average(test_mrr), np.var(test_mrr))